In [1]:
!pip install lightgbm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd # type: ignore
import lightgbm as lgb  # type: ignore
import warnings
warnings.filterwarnings("ignore")

train_df = pd.read_csv('data/UNSW_NB15_training.csv')
test_df = pd.read_csv('data/Book1.csv')

print(lgb.__version__)

4.5.0


In [ ]:
features = ['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 
            'rate', 'sload', 'dload', 'sloss', 'dloss', 'swin', 'dwin', 
            'is_sm_ips_ports']
target = 'attack_cat_number'  # Adjust if your target column has a different name

# Identify categorical features
cat_features = ['proto', 'service', 'state']
for col in cat_features:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Split into X (features) and y (labels)
X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

In [4]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_features)

# Define model parameters
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'num_class': 10,
    'verbose': -1
}

# Train model
model = lgb.train(
    params,
    train_data,
    num_boost_round=200,  # Ensure this is before early stopping
    valid_sets=[test_data],
    valid_names=["validation"],  # Give validation set a name
    # early_stopping_rounds=50  # This should be placed after valid_sets
)


In [5]:
from sklearn.metrics import accuracy_score, classification_report

# Predict class probabilities
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = y_pred_probs.argmax(axis=1)  # Since it's a multiclass problem

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.0000
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00    2000.0
           2       0.00      0.00      0.00    1746.0
           3       0.00      0.00      0.00   12264.0
           4       0.00      0.00      0.00   33393.0
           5       0.00      0.00      0.00   18184.0
           6       0.00      0.00      0.00   40000.0
           7       0.00      0.00      0.00   56000.0
           8       0.00      0.00      0.00   10491.0
           9       0.00      0.00      0.00    1133.0
          10       0.00      0.00      0.00     130.0

    accuracy                           0.00  175341.0
   macro avg       0.00      0.00      0.00  175341.0
weighted avg       0.00      0.00      0.00  175341.0

